In [1]:
import pandas as pd
import sys
from data_reader import fes_reader, perf_reader
from tqdm import tqdm

In [2]:
# конвертация путей файлов в зависимости от системы
def replace_slash(file_path):
    platform = sys.platform
    slash_map = {'win32': '\\',
                'cygwin': '\\',
                'darwin': '/',
                'linux2': '/'}
    if platform not in slash_map.keys(): platform = 'linux2'
    return file_path.replace('\\', slash_map[platform])

In [3]:
perf_path = replace_slash('input_data\\Перфорация.xlsx')
fes_path = replace_slash('input_data\\ФЕС.xlsx')

In [4]:
perf_ints = perf_reader(perf_path)
fes_df = fes_reader(fes_path)
SOIL_CUT = 60

In [5]:
fes_df

,well,top,bot,нас-ть,пор-ть,прон-ть,soil
0,10063,1719.199951,1722.799951,26.0,18.799999,266.600006,78.400002
1,10063,1722.799951,1733.800049,NaN,NaN,NaN,NaN
2,10063,1733.800049,1738.800049,26.0,17.299999,184.199997,74.699997
3,10063,1738.800049,1741.599976,NaN,NaN,NaN,NaN
4,10063,1741.599976,1742.799976,26.0,16.500000,122.000000,79.000000
...,...,...,...,...,...,...,...
2583,689,1580.000000,1581.199951,NaN,NaN,NaN,NaN
2584,689,1581.199951,1582.399951,26.0,13.100000,30.000000,74.000000
2585,689,1582.399951,1590.400024,NaN,NaN,NaN,NaN
2586,689,1590.400024,1593.800025,26.0,18.900000,230.000000,82.000000


In [8]:
def is_perf(top, bot, ints):
    if ints is None:
        return False
    for int_perf in ints:
        if (top <= int_perf['top'] <= bot) or (top <= int_perf['bot'] <= bot):
            return True
        elif (top >= int_perf['top']) and (int_perf['bot'] >= bot):
            return True
    return False


In [11]:
lost_layers = pd.DataFrame(columns=['well', 'top', 'bot', 'soil', 'is_perf'])
for well in tqdm(fes_df['well'].unique()):
    well_df = fes_df[fes_df['well'] == well][['well', 'top', 'bot', 'soil']]
    well_df.dropna(inplace=True)
    ints = perf_ints.get(well)
    well_df['is_perf'] = well_df.apply(lambda x: is_perf(x['top'], x['bot'], ints), axis=1)
    non_perf = well_df[(well_df['is_perf'] == False) & (well_df['soil'] > SOIL_CUT)]
    lost_layers = lost_layers.append(non_perf, ignore_index=True)

100%|██████████| 144/144 [00:00<00:00, 166.34it/s]


In [12]:
lost_layers


,well,top,bot,soil,is_perf
0,10063,1754.599976,1755.799976,61.000000,False
1,10063,1767.199951,1767.999951,62.000000,False
2,10066,1617.199951,1619.399951,76.500000,False
3,10066,1682.599976,1683.399976,74.099998,False
4,10066,1684.400024,1686.200024,76.300003,False
...,...,...,...,...,...
246,32488,1743.400024,1744.000024,76.599998,False
247,32488,1745.199951,1745.999951,75.800003,False
248,418,1663.599976,1665.999976,80.000000,False
249,522,1588.000000,1590.800000,70.000000,False
